# **PH 431: *Exploration of Electrodynamic Simulations Project*** by Dan Jang

## **Abstract**
Based on concepts explored in *PH 431: Electricity & Magnetism I* and from our textbook, *Introduction to Electrodynamics* by David J. Griffiths [[1]](https://www.cambridge.org/highereducation/books/introduction-to-electrodynamics/3AB220820DBB628E5A43D52C4B011ED4#overview), we created a computational physics project based in Python for exploring the simulation of various electrodynamics phenomena, particularly in the modeling and visualization of *electromagnetic fields*, *potentials*, and *point charges*.

Our exploration project was primarily based on a deconstructed, piece-by-piece implementation of these electrodynamics' topics in electromagnetism, where a *Python*-based, *Project Jupyter Notebook* [[2]](https://jupyter.org) was used to highlight particular parts & pieces of our project, with specific examples of electromagnetic phenomena and situations.

Our project primarily features the *PyCharge* [[3]](http://arxiv.org/abs/2107.12437) and *PyGame* [[4]](https://www.pygame.org/wiki/about) libraries – as well as the implementation of other libraries and visual or mathematical methods of computational physics.

The goal of our project was to help provide an elementary, or otherwise easy-to-under introduction to electromagnetism, particularly pertaining to the 3D representation of electromagnetic fields & potentials and moving point charges in those fields and potentials.

## Part 1: ***Initialization & Setup***

In [ ]:
##### PH 431: Exploration of Electrodynamic Simulations Project by Dan J.
#### Initialization of Project Libraries, Constants, & Jupyter Notebook

## Part 2: ***Electric Fields***

In [ ]:
##### PH 431: Exploration of Electrodynamic Simulations Project by Dan J.
#### 

## Part 3: ***Electric Potentials***

In [ ]:
##### PH 431: Exploration of Electrodynamic Simulations Project by Dan J.
#### 

## Part 4: ***Point Charges***

In [ ]:
##### PH 431: Exploration of Electrodynamic Simulations Project by Dan J.
#### 

## References

1.	Griffiths DJ. Introduction to electrodynamics. 4 ed. Cambridge: Cambridge university press; 2017.
2.	Project Jupyter [Internet]. Available from: https://jupyter.org.
3.	Filipovich MJ, Hughes S. PyCharge: An open-source Python package for self-consistent electrodynamics simulations of Lo-rentz oscillators and moving point charges. Computer Physics Communications [Internet]. 2022 May;274:108291. Available from: http://arxiv.org/abs/2107.12437.
4.	About - pygame wiki [Internet]. Available from: https://www.pygame.org/wiki/about.
